# Get Dataset

!pip install roboflow

In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="pdMgovjBJVklsz3rr36u")
project = rf.workspace("tennisanalysis-5huqh").project("tennis-ball-detection-final")
version = project.version(1)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to tennis-ball-detection-final-1 in yolov5pytorch:: 100%|██████████| 4681/4681 [00:00<00:00, 8287.09it/s]


In [4]:
import shutil
import os 
import random

In [3]:
shutil.move("tennis-ball-detection-final-1/train", 
            "tennis-ball-detection-final-1/tennis-ball-detection-final-1/train") 

shutil.move("tennis-ball-detection-final-1/valid", 
            "tennis-ball-detection-final-1/tennis-ball-detection-final-1/valid") 

shutil.move("tennis-ball-detection-final-1/test", 
            "tennis-ball-detection-final-1/tennis-ball-detection-final-1/test") 

'tennis-ball-detection-final-1/tennis-ball-detection-final-1/test'

In [7]:
os.getcwd()

'/Users/stan/Desktop/tennis_analysis/training'

In [8]:
import os
import shutil
import random

# Define paths
train_images_dir = 'tennis-ball-detection-final-1/tennis-ball-detection-final-1/train/images'
train_labels_dir = 'tennis-ball-detection-final-1/tennis-ball-detection-final-1/train/labels'

valid_images_dir = 'tennis-ball-detection-final-1/tennis-ball-detection-final-1/valid/images'
valid_labels_dir = 'tennis-ball-detection-final-1/tennis-ball-detection-final-1/valid/labels'

test_images_dir = 'tennis-ball-detection-final-1/tennis-ball-detection-final-1/test/images'
test_labels_dir = 'tennis-ball-detection-final-1/tennis-ball-detection-final-1/test/labels'

# Get list of filenames in each set
train_images = os.listdir(train_images_dir)
valid_images = os.listdir(valid_images_dir)
test_images = os.listdir(test_images_dir)

# Calculate total number of images
total_images = len(train_images) + len(valid_images) + len(test_images)

# Desired numbers
desired_train = int(0.7 * total_images)
desired_valid = int(0.2 * total_images)
desired_test = int(0.1 * total_images)

# Function to move files
def move_files(image_list, source_img_dir, source_lbl_dir, target_img_dir, target_lbl_dir, num_to_move):
    files_moved = []
    files_to_move = random.sample(image_list, num_to_move)
    for file_name in files_to_move:
        try:
            # Move image
            shutil.move(os.path.join(source_img_dir, file_name), os.path.join(target_img_dir, file_name))
            
            # Move corresponding label
            label_name = file_name.replace('.jpg', '.txt')
            shutil.move(os.path.join(source_lbl_dir, label_name), os.path.join(target_lbl_dir, label_name))
            
            files_moved.append(file_name)
        except FileNotFoundError:
            print(f"File not found: {file_name}. Skipping.")
            
    # Remove moved files from the original list
    for file_name in files_moved:
        image_list.remove(file_name)

# Adjust the dataset by moving images from train to valid/test
if len(train_images) > desired_train:
    excess_train = len(train_images) - desired_train
    if len(valid_images) < desired_valid:
        valid_needed = min(desired_valid - len(valid_images), excess_train)
        move_files(train_images, train_images_dir, train_labels_dir, valid_images_dir, valid_labels_dir, valid_needed)
        excess_train -= valid_needed
    if len(test_images) < desired_test:
        test_needed = min(desired_test - len(test_images), excess_train)
        move_files(train_images, train_images_dir, train_labels_dir, test_images_dir, test_labels_dir, test_needed)

# If necessary, balance validation and test sets
if len(valid_images) > desired_valid:
    excess_valid = len(valid_images) - desired_valid
    move_files(valid_images, valid_images_dir, valid_labels_dir, test_images_dir, test_labels_dir, excess_valid)
elif len(test_images) > desired_test:
    excess_test = len(test_images) - desired_test
    move_files(test_images, test_images_dir, test_labels_dir, valid_images_dir, valid_labels_dir, excess_test)

print("Dataset balancing complete.")

Dataset balancing complete.


In [ ]:
!yolo task=detect mode=train model=yolov516u.pt data={dataset.location}/data.yaml epochs=100 imgsz=640 